[[第 9.0 章 MPI入门]]

---

## 2.1 基本概念

### 点对点通信模型

- **发送端**：使用 `MPI_Send` 将数据从一个进程发送到另一个进程。
- **接收端**：使用 `MPI_Recv` 接收来自发送端的数据。
- **同步通信**：发送和接收操作通常是阻塞的，必须配对完成。
- **非阻塞通信**：可以使用 `MPI_Isend` 和 `MPI_Irecv` 实现异步通信。

---

## 2.2 **同步通信**

1. **MPI_Send**
    
    用于发送数据到目标进程：
    
    ```C
    int MPI_Send(const void *buf, int count, MPI_Datatype datatype,
    						int dest, int tag, MPI_Comm comm);
    ```
    
    参数解释：
    
    - `buf`：要发送的数据缓冲区。
    - `count`：发送的数据数量。
    - `datatype`：数据类型（如 `MPI_INT`）。
    - `dest`：目标进程的编号（Rank）。
    - `tag`：消息标签，用于区分消息。
    - `comm`：通信域（通常是 `MPI_COMM_WORLD`）。
2. **MPI_Recv**
    
    用于从指定进程接收数据：
    
    ```C
    int MPI_Recv(void *buf, int count, MPI_Datatype datatype,
    						int source, int tag, MPI_Comm comm, MPI_Status *status);
    
    ```
    
    参数解释：
    
    - `buf`：用于接收数据的缓冲区。
    - `count`：接收的数据最大数量。
    - `datatype`：数据类型。
    - `source`：数据来源进程的编号。
    - `tag`：消息标签。
    - `status`：用于存储接收信息（如消息长度、来源等）。
        - `MPI_Status` 提供了消息通信的核心信息，主要包含以下内容：
            - 消息来源（`MPI_SOURCE`）
            - 消息标签（`MPI_TAG`）
            - 错误状态（`MPI_ERROR`）
        - 如果你确定不会使用 `MPI_Status` 中的信息（例如消息来源、标签或错误状态），可以用 `MPI_STATUS_IGNORE` 替代需要传递 `MPI_Status` 的参数，减少不必要的变量声明和检查。

---

## 2.3 示例代码：同步通信

以下程序实现了两个进程之间的数据发送和接收：

```C
\#include <mpi.h>
\#include <stdio.h>

int main(int argc,char *argv[]){
	int rank;
	MPI_Init(&argc,&argv);

	MPI_Comm_rank(MPI_COMM_WORLD,&rank);

	if (rank == 0){
		int data = 41;
		MPI_Send(&data,1,MPI_INT,1,1,MPI_COMM_WORLD);
		printf("rank 0 send message 41 to 1\n");
		int data2 = 42;
		MPI_Send(&data2,1,MPI_INT,2,2,MPI_COMM_WORLD);
		printf("rank 0 send message 42 to  2\n");
	}
	if (rank == 1){
		int receive_data;
		MPI_Recv(&receive_data,1,MPI_INT,0,1,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
		printf("rank 1 receive message %d from 0\n",receive_data);
	}
	if (rank == 2){
		int receive_data;
		MPI_Recv(&receive_data,1,MPI_INT,0,2,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
		printf("rank 1 receive message %d from 0\n",receive_data);
	}
	MPI_Finalize();
	return 0;
}
```

---

## 2.4 非阻塞通信

### 1. **非阻塞发送：**`**MPI_Isend**`

非阻塞发送操作允许发送进程在发送消息的同时继续执行其他任务。

### 函数原型

```C
int MPI_Isend(const void *buf, int count, MPI_Datatype datatype,
              int dest, int tag, MPI_Comm comm, MPI_Request *request);
```

### 参数详解

- `**buf**`：指向要发送的数据的缓冲区。
- `**count**`：要发送的数据数量。
- `**datatype**`：数据类型（如 `MPI_INT`, `MPI_FLOAT`）。
- `**dest**`：目标进程的编号（Rank）。
- `**tag**`：消息标签，用于区分消息。
- `**comm**`：通信域（通常为 `MPI_COMM_WORLD`）。
- `**request**`：返回一个 `MPI_Request` 对象，用于后续的完成检测或等待操作。

---

### 2. **非阻塞接收：**`**MPI_Irecv**`

非阻塞接收操作允许接收进程在接收消息的同时继续执行其他任务。

### 函数原型

```C
int MPI_Irecv(void *buf, int count, MPI_Datatype datatype,
              int source, int tag, MPI_Comm comm, MPI_Request *request);
```

### 参数详解

- `**buf**`：指向接收数据的缓冲区。
- `**count**`：接收的数据最大数量。
- `**datatype**`：数据类型（如 `MPI_INT`, `MPI_FLOAT`）。
- `**source**`：消息来源的进程编号（`MPI_ANY_SOURCE` 表示接收任意来源的消息）。
- `**tag**`：消息标签（`MPI_ANY_TAG` 表示接收任意标签的消息）。
- `**comm**`：通信域（通常为 `MPI_COMM_WORLD`）。
- `**request**`：返回一个 `MPI_Request` 对象，用于后续的完成检测或等待操作。


### 3. **检查或等待非阻塞通信完成**

非阻塞通信需要通过额外的函数来检查或等待通信操作的完成。

### 3.1 `**MPI_Test**`

检查非阻塞通信是否完成。

```C
int MPI_Test(MPI_Request *request, int *flag, MPI_Status *status);
```

- `**request**`：非阻塞操作的请求句柄。
- `**flag**`：标志变量，通信完成时返回 `1`，否则返回 `0`。
- `**status**`：通信状态，包含来源、标签和错误码的信息。

---

### 3.2 `**MPI_Wait**`

阻塞等待非阻塞通信完成。

```C
int MPI_Wait(MPI_Request *request, MPI_Status *status);
```

- `**request**`：非阻塞操作的请求句柄。
- `**status**`：通信状态。

---

### 3.3 `**MPI_Waitall**`

等待多个非阻塞通信完成。

```C
int MPI_Waitall(int count, MPI_Request array_of_requests[], MPI_Status array_of_statuses[]);
```

- `**count**`：请求数量。
- `**array_of_requests**`：请求数组。
- `**array_of_statuses**`：状态数组。

---

### 3.4 `**MPI_Testall**`

检查多个非阻塞通信是否完成。

```C
int MPI_Testall(int count, MPI_Request array_of_requests[], int *flag, MPI_Status array_of_statuses[]);
```

- `**count**`：请求数量。
- `**array_of_requests**`：请求数组。
- `**flag**`：标志变量，所有请求完成时为 `1`，否则为 `0`。
- `**array_of_statuses**`：状态数组。

---

## 2.5 示例代码:非阻塞通信

非阻塞通信允许程序在发送或接收数据时继续执行其他操作，可以通过 `MPI_Isend` 和 `MPI_Irecv` 实现。

In [ ]:
\#include <mpi.h>
\#include <stdio.h>

int main(int argc, char *argv[]) {
    int rank;
    MPI_Init(&argc, &argv);

    // 获取当前进程编号
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    if (rank == 0) {
        int data = 99;
        MPI_Request request;
        MPI_Isend(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &request);
        printf("Rank 0 sent data %d to Rank 1 (non-blocking)\n", data);
    } else if (rank == 1) {
        int received_data;
        MPI_Request request;
        MPI_Status status;
        MPI_Irecv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &request);

        // 等待接收完成
        MPI_Wait(&request, &status);
        printf("Rank 1 received data %d from Rank 0 (non-blocking)\n", received_data);
    }

    MPI_Finalize();
    return 0;
}

---